In [1]:
import cPickle
from PIL import Image
import os
import sys
import numpy as np
from matplotlib.pyplot import imshow
import tensorflow as tf
import random


data_folder = 'cifar-10-batches-py'
train_batch_1 = os.path.join(data_folder, 'data_batch_1')
train_batch_2 = os.path.join(data_folder, 'data_batch_2')
train_batch_3 = os.path.join(data_folder, 'data_batch_3')
train_batch_4 = os.path.join(data_folder, 'data_batch_4')
train_batch_5 = os.path.join(data_folder, 'data_batch_5')
test_batch = os.path.join(data_folder, 'test_batch')

def read_batch(filename):
    with open(filename, 'r') as f:
        data = cPickle.load(f)
    return data

train_filenames = [train_batch_1, train_batch_2, train_batch_3, train_batch_4, train_batch_5]
train_data_arr = []
train_labels_arr = []
for filename in train_filenames:
    d = read_batch(filename)
    data = d['data']
    labels = d['labels']
    train_data_arr.append(data)
    train_labels_arr.append(np.asarray(labels, dtype=np.int32))
train_data = np.vstack(train_data_arr)
train_labels = np.hstack(train_labels_arr)
print train_data.shape
print train_labels.shape

test_d = read_batch(test_batch)
test_data = test_d['data']
test_labels = np.asarray(test_d['labels'], dtype=np.int32)
print test_data.shape
print test_labels.shape

(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


In [3]:
class CifarData:
    def __init__(self, data, labels, need_shuffle=True):
        self._data = data
        self._labels = labels
        assert len(data) == len(labels)
        self._indicator = 0
        self._need_shuffle = need_shuffle
        if self._need_shuffle:
            self._shuffle_data()
    
    def _shuffle_data(self):
        p = np.random.permutation(len(self._data))
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        end_indicator = self._indicator + batch_size
        if end_indicator > len(self._data):
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("determine dataset has no more examples")
        if end_indicator > len(self._data):
            raise Exception("batch size is larger than all example number")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels

In [4]:
train_data_iter = CifarData(train_data, train_labels)
test_data_iter = CifarData(test_data, test_labels)
batch_data, batch_labels = train_data_iter.next_batch(5)
print batch_data, batch_labels

[[ 67  96 100 ...  74  73  74]
 [ 30  34  36 ...  88  85  93]
 [147 137 152 ... 181 181 222]
 [ 64  66  84 ...  26  23  23]
 [  2   2   1 ...   1   2   1]] [6 5 6 4 3]


In [5]:
def cnn_model():
    with tf.name_scope("inputs"):
        is_training = tf.placeholder(tf.bool, name="is_training")
        x = tf.placeholder(tf.float32, [None, 3072])
        x_image = tf.reshape(x, [-1, 3, 32, 32])
        x_image = tf.transpose(x_image, perm=[0, 2, 3, 1])
        y = tf.placeholder(tf.int64, [None])
    with tf.name_scope("model"):
        conv1 = tf.layers.conv2d(inputs=x_image,
                                 filters=32,
                                 kernel_size=[5,5],
                                 padding='same',
                                 activation=tf.nn.relu)
        pool1 = tf.layers.max_pooling2d(inputs=conv1,
                                        pool_size=[2,2],
                                        strides=2)
        
        conv2 = tf.layers.conv2d(inputs=pool1,
                                 filters=64,
                                 kernel_size=[5,5],
                                 padding='same',
                                 activation=tf.nn.relu)
        pool2 = tf.layers.max_pooling2d(inputs=conv2,
                                        pool_size=[2,2],
                                        strides=2)
        
        conv3 = tf.layers.conv2d(inputs=pool2,
                                 filters=64,
                                 kernel_size=[5,5],
                                 padding='same',
                                 activation=tf.nn.relu)
        pool3 = tf.layers.max_pooling2d(inputs=conv3,
                                        pool_size=[2,2],
                                        strides=2)
        
        pool3_flatten = tf.layers.flatten(pool3)
        dense = tf.layers.dense(inputs=pool3_flatten,
                                units=128,
                                activation=tf.nn.relu)
        dropout = tf.layers.dropout(inputs=dense,
                                    rate=0.1,
                                    training=is_training)
        logits = tf.layers.dense(inputs=dropout,
                                units=10)
    return x, y, is_training, logits

In [9]:
x, y, is_training, logits = cnn_model()
with tf.name_scope('metrics'):
    cross_entropy = tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits))
    correct_prediction = tf.equal(tf.argmax(logits, 1), y)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(cross_entropy)

init = tf.global_variables_initializer()

batch_size = 20
train_steps = 500
test_steps_each = 100
with tf.Session() as sess:
    sess.run(init)
    train_data_iter = CifarData(train_data, train_labels)
    for i in range(train_steps):
        batch_data, batch_labels = train_data_iter.next_batch(batch_size)
        cross_entropy_val, acc_val, _ = sess.run([cross_entropy, accuracy, train_op],
                                                 feed_dict = {
                                                     x: batch_data,
                                                     y: batch_labels,
                                                     is_training: True
                                                 })
        if (i+1) % 50 == 0:
            print '[Train] Step %4d: ce: %4.5f, acc: %4.5f' % (i+1, cross_entropy_val, acc_val)
        
    test_data_iter = CifarData(test_data, test_labels, need_shuffle=False)
    acc_vals = []
    ce_vals = []
    for j in range(test_steps_each):
        test_batch_data, test_batch_labels = test_data_iter.next_batch(batch_size)
        ce_val, acc_val = sess.run([cross_entropy, accuracy], 
                                   feed_dict = {
                                       x: test_batch_data,
                                       y: test_batch_labels,
                                       is_training: False
                                   })
        acc_vals.append(acc_val)
        ce_vals.append(ce_val)
    print '[Test ] Overall: ce: %4.5f, acc: %4.5f' % (np.mean(ce_vals), np.mean(acc_vals))

[Train] Step   50: ce: 2.30084, acc: 0.15000
[Train] Step  100: ce: 2.14018, acc: 0.15000
[Train] Step  150: ce: 1.87919, acc: 0.20000
[Train] Step  200: ce: 1.82605, acc: 0.20000
[Train] Step  250: ce: 1.80814, acc: 0.45000
[Train] Step  300: ce: 1.87352, acc: 0.25000
[Train] Step  350: ce: 1.70463, acc: 0.35000
[Train] Step  400: ce: 2.01196, acc: 0.40000
[Train] Step  450: ce: 1.87371, acc: 0.40000
[Train] Step  500: ce: 2.02945, acc: 0.15000
[Test ] Overall: ce: 1.76995, acc: 0.34500
